In [10]:
import pandas as pd
import geopy
import geopy.distance
from shapely.geometry import Polygon, LineString, Point 
from math import cos  
from math import sin  
from math import pi
from datetime import datetime, timedelta

In [32]:
met_data = pd.read_excel('Meteorological Data.xlsx')
sensor_data = pd.read_excel('Sensor Data.xlsx')

In [33]:
met_data_1 = met_data.copy()
met_data_2 = met_data.copy()

met_data_1['Date'] = met_data_1['Date'] + timedelta(hours=1)
met_data_2['Date'] = met_data_2['Date'] + timedelta(hours=2)

met_data = met_data.append([met_data_1, met_data_2]).sort_values('Date').reset_index(drop=True)

In [34]:
merged_data = sensor_data.merge(met_data[['Date', 'Wind Direction', 'Wind Speed (m/s)']].rename(columns={'Date': 'Date Time '}), on='Date Time ')
merged_data

ValueError: You are trying to merge on object and datetime64[ns] columns. If you wish to proceed you should use pd.concat

In [5]:
for i, row in merged_data.iterrows():
    if row['Wind Direction'] <= 180:
        merged_data.loc[i, 'actual_direction'] = row['Wind Direction'] + 180
    else:
        merged_data.loc[i, 'actual_direction'] = row['Wind Direction'] - 180

In [6]:
sensor_loc_map = {1: [62,21],
                2: [66,35],
                3: [76,41],
                4: [88,45],
                5: [103,43],
                6: [102,22],
                7: [89,3],
                8: [74,7],
                9: [119,42]}

In [7]:
merged_data['Coordinates'] = merged_data['Monitor'].map(sensor_loc_map)
roadrunner = Point(89, 27)
kasios = Point(90, 21)
radiance = Point(109, 26)
indigo = Point(120, 22)

In [8]:
def factory_distances(x, y, angle):  

    angle = angle * pi / 180.0  
    line = LineString([(x, y), (x + 10000 * sin(angle), y + 10000 * cos(angle))]) 
    
    roadrunner_dist = roadrunner.distance(line)
    kasios_dist = kasios.distance(line)
    radiance_dist = radiance.distance(line)
    indigo_dist = indigo.distance(line)
    
    return roadrunner_dist, kasios_dist, radiance_dist, indigo_dist

In [9]:
for i, row in merged_data.iterrows():
    roadrunner_dist, kasios_dist, radiance_dist, indigo_dist = factory_distances(row['Coordinates'][0], row['Coordinates'][1], row['actual_direction'])
    merged_data.loc[i, 'roadrunner_dist'] = roadrunner_dist
    merged_data.loc[i, 'kasios_dist'] = kasios_dist
    merged_data.loc[i, 'radiance_dist'] = radiance_dist
    merged_data.loc[i, 'indigo_dist'] = indigo_dist
    
    l = [roadrunner_dist, kasios_dist, radiance_dist, indigo_dist]
    indices_order = [l.index(x) for x in sorted(l)]
    merged_data.loc[i, 'highest_contributor'] = indices_order[0]
    merged_data.loc[i, 'second_highest_contributor'] = indices_order[1]
    merged_data.loc[i, 'third_highest_contributor'] = indices_order[2]
    merged_data.loc[i, 'fourth_highest_contributor'] = indices_order[3]

KeyboardInterrupt: 

In [ ]:
factory_map = {0: 'Roadrunner Fitness Electronics',
                1: 'Kasios Office Furniture',
                2: 'Radiance ColourTek',
                3: 'Indigo Sol Boards',}

In [ ]:
merged_data['highest_contributor'] = merged_data['highest_contributor'].map(factory_map)
merged_data['second_highest_contributor'] = merged_data['second_highest_contributor'].map(factory_map)
merged_data['third_highest_contributor'] = merged_data['third_highest_contributor'].map(factory_map)
merged_data['fourth_highest_contributor'] = merged_data['fourth_highest_contributor'].map(factory_map)

In [ ]:
merged_data

In [ ]:
merged_data.to_csv('complete_data.csv', index=False)

In [65]:
merged_data = pd.read_csv("complete_data.csv")
temp = merged_data[merged_data["Chemical"] == "Methylosmolene"][merged_data["Monitor"] == 1]
temp.index.name = "index"
temp[["Wind Direction", "Wind Speed (m/s)"]].rename(columns={'Wind Direction': 'direction', "Wind Speed (m/s)" : "speed"}).to_csv("windData.csv")
temp

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (8,9,10,11) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


,Chemical,Monitor,Date Time,Reading,Wind Direction,Wind Speed (m/s),actual_direction,Coordinates,roadrunner_dist,kasios_dist,radiance_dist,indigo_dist,highest_contributor,second_highest_contributor,third_highest_contributor,fourth_highest_contributor
index,,,,,,,,,,,,,,,,
11,Methylosmolene,1,2016-04-01 00:00:00,0.417423,190.5,4.0,10.5,"[62, 21]",25.4545,27.5311,45.3018,56.8465,Roadrunner Fitness Electronics,Kasios Office Furniture,Radiance ColourTek,Indigo Sol Boards
49,Methylosmolene,1,2016-04-01 01:00:00,0.381961,190.5,4.0,10.5,"[62, 21]",25.4545,27.5311,45.3018,56.8465,Roadrunner Fitness Electronics,Kasios Office Furniture,Radiance ColourTek,Indigo Sol Boards
105,Methylosmolene,1,2016-04-01 02:00:00,0.037844,190.5,4.0,10.5,"[62, 21]",25.4545,27.5311,45.3018,56.8465,Roadrunner Fitness Electronics,Kasios Office Furniture,Radiance ColourTek,Indigo Sol Boards
124,Methylosmolene,1,2016-04-01 03:00:00,0.353672,203.3,5.0,23.3,"[62, 21]",22.4248,25.7165,41.1893,52.8743,Roadrunner Fitness Electronics,Kasios Office Furniture,Radiance ColourTek,Indigo Sol Boards
166,Methylosmolene,1,2016-04-01 04:00:00,0.284877,203.3,5.0,23.3,"[62, 21]",22.4248,25.7165,41.1893,52.8743,Roadrunner Fitness Electronics,Kasios Office Furniture,Radiance ColourTek,Indigo Sol Boards
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75920,Methylosmolene,1,2016-12-31 19:00:00,0.241271,177.0,3.3,357.0,"[62, 21]",27.277,28,47.1973,58.0086,Roadrunner Fitness Electronics,Kasios Office Furniture,Radiance ColourTek,Indigo Sol Boards
75961,Methylosmolene,1,2016-12-31 20:00:00,0.099621,177.0,3.3,357.0,"[62, 21]",27.277,28,47.1973,58.0086,Roadrunner Fitness Electronics,Kasios Office Furniture,Radiance ColourTek,Indigo Sol Boards
75994,Methylosmolene,1,2016-12-31 21:00:00,0.134625,161.0,3.5,341.0,"[62, 21]",27.6586,28,47.2652,58.0086,Roadrunner Fitness Electronics,Kasios Office Furniture,Radiance ColourTek,Indigo Sol Boards
